In [1]:
import bz2
import json
import requests as rq
import time

In [50]:
import lzma

In [46]:
# dump = "/datasets_1/sagarj/reddit_all_2017/RC_2017-02.bz2"
dump = "/10TBdrive/datashare/Reddit/submissions/RS_2017-12.xz"

In [51]:
# openedDump = bz2.BZ2File(dump, "r")
openedDump = lzma.open(dump)

In [52]:
count = 0
for line in openedDump:
    if count > 10:
        break
    print(repr(line))
    count+=1

b'{"archived":false,"author":"MortallyHolyRunaway","author_flair_css_class":null,"author_flair_text":null,"brand_safe":true,"contest_mode":false,"created_utc":1512086408,"distinguished":null,"domain":"v.redd.it","edited":false,"gilded":0,"hidden":false,"hide_score":false,"id":"7gqv9t","is_crosspostable":false,"is_reddit_media_domain":true,"is_self":false,"is_video":true,"link_flair_css_class":"video","link_flair_text":"Video","locked":false,"media":{"reddit_video":{"dash_url":"https://v.redd.it/mfps4a7za7101/DASHPlaylist.mpd","duration":49,"fallback_url":"https://v.redd.it/mfps4a7za7101/DASH_1_2_M","height":360,"hls_url":"https://v.redd.it/mfps4a7za7101/HLSPlaylist.m3u8","is_gif":true,"scrubber_media_url":"https://v.redd.it/mfps4a7za7101/DASH_600_K","transcoding_status":"completed","width":640}},"media_embed":{},"num_comments":0,"num_crossposts":0,"over_18":false,"parent_whitelist_status":"all_ads","permalink":"/r/cats/comments/7gqv9t/my_little_rose_trying_to_get_into_my_bedroom/","pin

In [54]:
def getJson(url):
    session_headers = {
    'Host': 'www.reddit.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    #'Referer': 'http://localhost:9000/notebooks/Reddit/Notebooks/SuicideWatchSubreddit.ipynb'
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en-US;q=0.8,en;q=0.6',
    'Cookie': "loid=00000000000gslzvdi.2.1507591930964.Z0FBQUFBQlozQWI3M243MTdFMUgzNWtQNzNVbzg5Z3NXYmxpLUdRTWVXdEpWQzBfbkxBSVNNVVF3dE1zQlN5cVMzeGtxZXlmT0JQY2JvTld2NWlVeGlGeFpkNnBYM2ZIYlNNR2xMWGFFS3hieDZpWEJ5OVlyTng0MUFZZjNYdjQ0ZURFNTFlSmlVRGw; edgebucket=T8nkTF0z7WPnNzlE8B; __gads=ID=ab9c1b0fcb7eaefc:T=1507591936:S=ALNI_Mao2tvEoyJzEDEuxxhfZBlOQHXlsA; eu_cookie_v2=3; _ga=GA1.2.1088810883.1507591931; __utma=55650728.1088810883.1507591931.1507912304.1507912553.14; __utmz=55650728.1507912553.14.11.utmcsr=reddit|utmccn=(not%20set)|utmcmd=hot|utmcct=comments; aa=1; _recentclicks2=t3_7cfzsx%2Ct3_7cfw08%2Ct3_7cdofu%2Ct3_7caog1%2Ct3_7cdvud; _recent_srs=t5_2qpzs%2Ct5_2qh1q%2Ct5_2qh49%2Ct5_2rtff%2Ct5_36buk%2Ct5_2r7yd%2Ct5_2rks3%2Ct5_3j2jr%2Ct5_2sumw%2Ct5_2xp2o; pc=r3; session_tracker=tPgx9U9yPt99jYRDqC.0.1510585492994.Z0FBQUFBQmFDYlNWcWZHclI5WWxDWW9kQ0dURk9FRU9BZjFJTURWZElzaldRbkROaEJUOWE1czRQUWxPUzUwOFFwY3R3ckI5VkNDUlhJc01NVEtEeHVWd25CdUpXWEI1X012RkJaS3cyNXdNbk5vOE9DSXp5ZlhuTzF3a2pxT2dLZ3laMDhtQlhib3k; initref=localhost"
    }

    client = rq.session()
    resp = client.get(url , headers=session_headers)
    if resp.status_code == 200:
        json_data = resp.json()
        return json_data
    else:
        print("Failed to get page")
        return None
def getSubredditId(json):
    if type(json) == dict:
        sub_id = json['data']['children'][0]['data']['subreddit_id']
        return sub_id
    else: 
        print("Invalid Format")
        return None

def makeSubUrl(name):
    return "https://www.reddit.com/r/" + name + "/.json"

def getSubIds(RedditList):
    subIds = {}
    for name in RedditList:
        json = getJson(makeSubUrl(name))
        print (getSubredditId(json))
        subIds[name] = getSubredditId(json)
    return subIds

def filterDumpbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit'] in subID_dict:
            if lineData['subreddit'] not in returnData:
                returnData[lineData['subreddit']] = []
            postId = lineData['link_id'].strip().split('_')[1]
            
            if postId not in returnData[lineData['subreddit']]:
                returnData[lineData['subreddit']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData


def filterSubmissionsbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit_id'] in subID_dict:
            if lineData['subreddit_id'] not in returnData:
                returnData[lineData['subreddit_id']] = []
            postId = lineData['id']
            
            if postId not in returnData[lineData['subreddit_id']]:
                returnData[lineData['subreddit_id']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData



def getSubmissionPermalinks(filePath , comments_Thresh = 20 , limit=None):
    import json
    import bz2
    import lzma
    
    ext = filePath.split('.')[-1]
    
    if ext == 'bz2':
        openedDump = bz2.BZ2File(filePath, "r")
    elif ext == 'xz':
        openedDump = lzma.open(dump)
    else:
        print ("Invalid file type")
    count = 0
    returnData = []
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if lineData['num_comments'] > comments_Thresh:
            if (limit!=None and count > limit):
                return returnData
            try:
                returnData.append(lineData['permalink'])
            except:
                print("Permalink not found, moving on!")
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData

In [22]:
subreddits = ["bipolar" , "BipolarReddit" , "BipolarSOs" , "Health" , "addiction", "depression","SuicideWatch","Anxiety","Chicago"] 

In [23]:
subID_dict = getSubIds(subreddits)

t5_2qnv4
t5_2qzru
t5_2zww6
t5_2qh9z
t5_2qwvc
t5_2qqqf
t5_2qpzs
t5_2qmij
t5_2qh2t


In [24]:
subID_dict

{'Anxiety': 't5_2qmij',
 'BipolarReddit': 't5_2qzru',
 'BipolarSOs': 't5_2zww6',
 'Chicago': 't5_2qh2t',
 'Health': 't5_2qh9z',
 'SuicideWatch': 't5_2qpzs',
 'addiction': 't5_2qwvc',
 'bipolar': 't5_2qnv4',
 'depression': 't5_2qqqf'}

In [57]:
# postData = filterDumpbySub(dump[:1000],subID_dict)
postData = getSubmissionPermalinks(dump[:1000],comments_Thresh= 2)

=============Parsing============
{'bipolar': 't5_2qnv4', 'BipolarReddit': 't5_2qzru', 'BipolarSOs': 't5_2zww6', 'Health': 't5_2qh9z', 'addiction': 't5_2qwvc', 'depression': 't5_2qqqf', 'SuicideWatch': 't5_2qpzs', 'Anxiety': 't5_2qmij', 'Chicago': 't5_2qh2t'}
Done parsing %d posts in %d seconds (0, 0.00078582763671875)
Done parsing %d posts in %d seconds (100000, 5.783618450164795)
Done parsing %d posts in %d seconds (200000, 5.64851975440979)
Done parsing %d posts in %d seconds (300000, 5.70309853553772)
Done parsing %d posts in %d seconds (400000, 5.7416112422943115)
Done parsing %d posts in %d seconds (500000, 5.6409711837768555)
Done parsing %d posts in %d seconds (600000, 5.804859638214111)
Done parsing %d posts in %d seconds (700000, 5.7712132930755615)
Done parsing %d posts in %d seconds (800000, 5.66406774520874)
Done parsing %d posts in %d seconds (900000, 5.907747268676758)
Done parsing %d posts in %d seconds (1000000, 6.20794939994812)
Done parsing %d posts in %d seconds (110

In [58]:
len(postData)

3499709

In [59]:
postData[1]

'/r/AskReddit/comments/7gqvc3/what_ungilded_commentpost_in_all_of_reddit_is/'

In [31]:
# import pickle as pkl
# with open("/datasets_1/sagarj/IoPPN_collab/MentalHealth_2017_posts.pkl",'w') as f:
#     pkl.dump(postData,f)

In [32]:
len(list(set(postData['depression'])))

KeyError: 'depression'

In [ ]:
# import pickle as pkl
# with open("ChangemyView2017.pkl",'w') as f:
#     pkl.dump(postData['changemyview'],f)

In [ ]:
# with open("Filtered_posts_2017.json",'w') as f:
#     json.dump(postData,f)